In [7]:
import copy
import json
import jsonschema
import os
import sys
import urllib

%cd ..

from src.rocrates_to_madmp import *

/Users/cl/Documents/masters/maDMP-rocrates-maDMP


In [8]:
def test_rocrate_dmp(dmp_header):
    if not 'contact' in dmp_header:
        print('argh')
        if 'contributor' in dmp_header:
            if not isinstance(dmp_header['contributor'], list):
                dmp_header['contributor'] = [copy.deepcopy(dmp_header['contributor'])]
            for contributor in dmp_header['contributor']:
                if not 'role' in contributor:
                    contributor['role'] = ['']
            dmp_header['contact'] = copy.deepcopy(dmp_header['contributor'][0])
            dmp_header['contact']['contact_id'] = dmp_header['contact'].pop('contributor_id')
    if not 'ethical_issues_exist' in dmp_header:
        dmp_header['ethical_issues_exist'] = 'unknown'
    if not 'language' in dmp_header:
        dmp_header['language'] = 'eng'
    for dataset in dmp_header['dataset']:
        for distribution in dataset['distribution']:
            if not 'data_access' in distribution:
                distribution['data_access'] = 'closed'


def rocrates_to_madmp(path, path_schema, mappings, dmp_identifier):
    print(os.listdir(path))
    
    manifest_dirs = [ name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name)) ]
    manifest_dirs.sort()
    if 'manifest.jsonld' in os.listdir(path):
        manifest_dirs.insert(0, '')
    
    dmp_id_type = get_identifer_type_for_dmp(dmp_identifier)
    dmp_header = {'title': 'DMP created from rocrates',
                 'description': 'a RDA-DMP-Common-Standard maDMP created from rocrate manifests',
                 'dmp_id': {
                     'identifier': dmp_identifier,
                     'type': dmp_id_type
                 }}
    
    dmp_from_rocrate = {'dmp': {}}
    datasets = []
    for manifest_dir in manifest_dirs:
        manifest = read_json_local(os.path.join(path, manifest_dir, 'manifest.jsonld'))
        dmp_header = {**dmp_header, **parse_mapping(manifest, dmp_header_to_dataset_mapping)}
        datasets.append(parse_mapping(manifest, dataset_mapping))
    dmp_header['dataset'] = datasets
    dmp_from_rocrate['dmp'] = dmp_header
    #print(json.dumps(dmp_from_rocrate, indent=4))
    # fix problems
    test_rocrate_dmp(dmp_header)
    
    # check schema
    schema = read_json_url(path_schema)
    valid = check_valid_dmp(dmp_from_rocrate, schema)
    
    write_json_local(os.path.join(path, 'dmp_from_rocrates.json'), dmp_from_rocrate)
    return datasets

In [9]:
path = 'examples/ex9-dmp-long/'
path = 'examples/simple-dataset-0.1.0'
path_schema = 'https://raw.githubusercontent.com/RDA-DMP-Common/RDA-DMP-Common-Standard/master/examples/JSON/JSON-schema/1.0/maDMP-schema-1.0.json'
dmp_identifier = "https://doi.org/10.0000/00.0.1234"
datasets = rocrates_to_madmp(path, path_schema, mappings, dmp_identifier)

['dmp_from_rocrates.json', 'manifest.jsonld']
argh
Checking if valid maDMP
VALID


In [4]:
datasets

[]